# `gen` API Examples

This notebook gives examples of how to use the `gen` command.

In [1]:
import guidance

gpt2 = guidance.llms.Transformers("gpt2", device=1)
gpt3 = guidance.llms.OpenAI("text-davinci-003")
gpt4 = guidance.llms.OpenAI("gpt-4")
guidance.llm = gpt2

## Basic usage

Below we have a program that includes a basic generation call using `gen`. There are two arguments passed to `gen` one positional argument and one keyword argument. The positional argument is the name of the program variable to store the generation in. The keyword argument `stop` is a string that tells `gen` when to stop generating (in this case we stop when generating a period).

In [4]:
program = guidance("""This is a sentence about {{gen "completion" stop="."}}""")
executed_program = program()

Stop program This is a sentence about how to make a good game

In [8]:
# can access the generated text as a variable in the executed program
executed_program["completion"]

'how to make a good game'

## `name` positional argument

The `name` argument is a string that represents where to store the results of the generation, if it is blank then the results are stored in the default variable `generated`.

In [9]:
# if we omit the variable_name the default of "generated"
out = guidance("""This is a sentence about {{gen stop="."}}""")()
out.variables()

Stop program This is a sentence about how to make a good game

{'llm': <guidance.llms._transformers.Transformers at 0x7f80382a8af0>,
 'generated': 'how to make a good game'}

In [12]:
# we can use dot notation to set properties of objects or elements of dictionaries
out = guidance("""This is a sentence about {{gen 'obj.completion' stop="."}}""")(obj={})
out["obj"]

Stop program This is a sentence about how to make a good game

{'completion': 'how to make a good game'}

## `stop` keyword argument

The `stop` argument can either be a string or a list of strings. If it is a string then it is the string that tells `gen` when to stop generating. If it is a list of strings then any of those strings will cause the generation to stop when they appear.

In [14]:
out = guidance("""This is a sentence about {{gen 'text' stop=[" the", " of", " a"]}}""")()
out["text"]

Stop program This is a sentence about how to make

'how to make'

## `stop_regex` keyword argument

The `stop_regex` argument is just like the `stop` argument but contains regular expressions instead of raw strings. This can be used to stop generation in a highly configurable way.

In [19]:
out = guidance("""This is a sentence about {{gen 'text' stop_regex=[" of[^a-z]", " the[^a-z]"]}}""")()
out["text"]

Stop program This is a sentence about how to make a good game.

The game is a game

'how to make a good game.\n\nThe game is a game'

In [7]:
import guidance

guidance.llm = guidance.llms.OpenAI("text-davinci-003", caching=False)

program = guidance("""Please solve the following word problem and call a calcuator with CALC(EQUATION) = ANSWER whenever you need to compute equations. For example: CALC((4+3) * 2) = 14.
Problem: {{problem}}
Reason step by step: {{gen 'text' stop_regex="CALC\\(.*?\\) =" max_tokens=100 save_stop_text=True}}""")
program(problem="Joe has ten apples and needs run 5 tests on each apple, if each test takes 7 minutes how long will this take Joe?")

Stop program Please solve the following word problem and call a calcuator with CALC(EQUATION) = ANSWER whenever you need to compute equations. For example: CALC((4+3) * 2) = 14.
Problem: Joe has ten apples and needs run 5 tests on each apple, if each test takes 7 minutes how long will this take Joe? 
Reason step by step: 

Joe has 10 apples and needs to run 5 tests on each apple. 

This means Joe needs to run 50 tests in total. 

Each test takes 7 minutes, so the total time it will take Joe is 50 tests multiplied by 7 minutes per test. 

CALC(50

In [8]:
# here we can see the stop text that was saved
out["text_stop_text"]

'CALC(50 * 7) ='

## `save_stop_text` keyword argument

The `save_stop_text` argument causes the gen command to save the text that caused it to stop generating. This is useful when you have a list of strings or a regular expression that you are using to stop generation and you want to know what exact string caused the generation to stop. If set to true it will save the stop text in a variable named `variable_name + "_stop_text"`, if set to a string it will save the stop text to the variable with that name.

In [24]:
# stop on any three letter word, and then print the word that was stopped on
out = guidance("""This is a sentence about {{gen 'text' stop_regex=" [a-z]{3}[^a-z]" save_stop_text=True temperature=1.0}}""")()
out.variables()

Stop program This is a sentence about the need

{'llm': <guidance.llms._transformers.Transformers at 0x7f80382a8af0>,
 'text': 'the need',
 'text_stop_text': ' for '}

In [25]:
# stop on any three letter word, and then print the word that was stopped on
out = guidance("""This is a sentence about {{gen 'text' stop_regex=" [a-z]{3}[^a-z]" save_stop_text="stop" temperature=1.0}}""")()
out["stop"]

Stop program This is a sentence about the need

' for '

## `max_tokens` keyword argument

In [26]:
out = guidance("""This is a sentence about {{gen 'text' max_tokens=10}}""")()
out["text"]

Stop program This is a sentence about how to make a good game.

The game

'how to make a good game.\n\nThe game'

## `n` keyword argument

The `n` argument controls how many generations to perform in a batch. If `n > 1` then only the first completion is used for future contex, and the rest are just stored in the variable.

In [27]:
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 temperature=1.0}}""")()
out["text"]

Stop program This is a sentence about <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_b7ea30a6034e4dc791a3a322bbc04485(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("b7ea30a6034e4dc791a3a322bbc04485_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("b7ea30a6034e4dc791a3a322bbc04485_" + (i+1)); if (!next_el) { next_el = document.getElementById("b7ea30a6034e4dc791a3a322bbc04485_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_b7ea30a6034e4dc791a3a322bbc04485(this);'>1/5 the 'rightness' of a different sort of person, self destruction in general as the 'our collective intelligence tells us what we call a 'human

["the 'rightness' of",
 'a different sort of person,',
 'self destruction in general as the',
 "'our collective intelligence tells us",
 "what we call a 'human"]

## `temperature` keyword argument

The `temperature` argument controls the sampling temperature and is passed directly to the LLM. By default `temperature` is set to 0 and the LLM does greedy sampling. This allows the LM calls to be cached and reused. If the `temperature` is set to a value greater than 0 then the LLM will do sampling and repeated calls in the same LM session (program execution) will lead to new generations (though re-runs of the same program will use caches for each of those calls).

In [36]:
# with a zero temperature, the generated text will be the same each time
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5}}""")()
out["text"]

Stop program This is a sentence about <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_cb68bb8fdd2e4da3ae89a7bbbdde397a(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("cb68bb8fdd2e4da3ae89a7bbbdde397a_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("cb68bb8fdd2e4da3ae89a7bbbdde397a_" + (i+1)); if (!next_el) { next_el = document.getElementById("cb68bb8fdd2e4da3ae89a7bbbdde397a_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_cb68bb8fdd2e4da3ae89a7bbbdde397a(this);'>1/5 how to make a good game how to make a good game how to make a good game how to make a good game how to make a good game

['how to make a good game',
 'how to make a good game',
 'how to make a good game',
 'how to make a good game',
 'how to make a good game']

In [ ]:
# with a non zero temperature, the generated text will be different each time
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 temperature=1.0}}""")()
out["text"]

Stop program This is a sentence about <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_aca97e5b0b454b599906c2dd97392b9d(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("aca97e5b0b454b599906c2dd97392b9d_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("aca97e5b0b454b599906c2dd97392b9d_" + (i+1)); if (!next_el) { next_el = document.getElementById("aca97e5b0b454b599906c2dd97392b9d_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_aca97e5b0b454b599906c2dd97392b9d(this);'>1/5 the 'rightness' of a different sort of person, self destruction in general as the 'our collective intelligence tells us what we call a 'human

["the 'rightness' of",
 'a different sort of person,',
 'self destruction in general as the',
 "'our collective intelligence tells us",
 "what we call a 'human"]

In [35]:
# but we still cache the results across calls (when caching is enabled) and so rerunning the program will give the same results as above
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 temperature=1.0}}""")()
out["text"]

Stop program This is a sentence about <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_7bc695ad015442609211600bb0e6f66c(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("7bc695ad015442609211600bb0e6f66c_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("7bc695ad015442609211600bb0e6f66c_" + (i+1)); if (!next_el) { next_el = document.getElementById("7bc695ad015442609211600bb0e6f66c_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_7bc695ad015442609211600bb0e6f66c(this);'>1/5 the 'rightness' of a different sort of person, self destruction in general as the 'our collective intelligence tells us what we call a 'human

["the 'rightness' of",
 'a different sort of person,',
 'self destruction in general as the',
 "'our collective intelligence tells us",
 "what we call a 'human"]

## `top_p` keyword argument

The `top_p` argument controls the proportion of the probability space used from sampling. By default it is 1.0, so we sample from the whole space. Note that setting `top_p` only matters if you have a non-zero `temperature` value.

In [39]:
# setting top_p to 0.4 will make the generated text less likely to chose low probability tokens by
# sampling from the top tokens that collectively have a probability of 0.4
out = guidance("""This is a sentence about {{gen 'text' n=5 max_tokens=5 top_p=0.4 temperature=1.0}}""")()
out["text"]

Stop program This is a sentence about <div style='background: rgba(255, 255, 255, 0.0); border-radius: 4px 0px 0px 4px; border: 1px solid rgba(0, 165, 0, 0.25); border-right: 0px; padding-left: 3px; padding-right: 3px; user-select: none; color: rgba(0, 165, 0, 0.25); display: inline; font-weight: normal; cursor: pointer' onClick='function cycle_50ade24abf3941faab4c289ee04ca8b7(button_el) { var i = 0; while (i < 50) { var el = document.getElementById("50ade24abf3941faab4c289ee04ca8b7_" + i); if (el.style.display == "inline") { el.style.display = "none"; var next_el = document.getElementById("50ade24abf3941faab4c289ee04ca8b7_" + (i+1)); if (!next_el) { next_el = document.getElementById("50ade24abf3941faab4c289ee04ca8b7_0"); } if (next_el) { next_el.style.display = "inline"; } break; } i += 1; } button_el.innerHTML = (((i+1) % 5) + 1) + "/" + 5;}cycle_50ade24abf3941faab4c289ee04ca8b7(this);'>1/5 a guy who is a bit how to make your own " how to make it work. a man who was arrested and the importance of having a healthy

['a guy who is a bit',
 'how to make your own "',
 'how to make it work.',
 'a man who was arrested and',
 'the importance of having a healthy']

## `logprobs` keyword argument

When `logprobs` is set to a number greater than 0 then the LLM will return the log probabilities of that many top tokens for each token in the text. These will be saved in a variable named `variable_name + "_logprobs"`.

In [3]:
# setting top_p to 0.4 will make the generated text less likely to chose low probability tokens by
# sampling from the top tokens that collectively have a probability of 0.4
out = guidance("""This is a sentence about {{gen 'text' max_tokens=5 logprobs=3}}""")()

# note that the odd character is due to the Transformers package tokenizer for GPT-2 representing the leading space as a special token
out["text_logprobs"]

Stop program This is a sentence about how to make a good game

[{'Ġhow': -2.4320783615112305,
  'Ġthe': -2.4620161056518555,
  'Ġa': -2.802485466003418},
 {'Ġto': -2.460637092590332,
  'Ġthe': -2.711819648742676,
  'ĠI': -3.0405778884887695},
 {'Ġmake': -2.956888198852539,
  'Ġget': -3.1506290435791016,
  'Ġuse': -3.2405643463134766},
 {'Ġa': -1.7699661254882812,
  'Ġyour': -2.6568679809570312,
  'Ġit': -2.8423233032226562},
 {'Ġgood': -3.4963274002075195,
  'Ġbetter': -4.585598945617676,
  'Ġgame': -4.620213508605957},
 {'Ġgame': -3.9762942790985107,
  'Ġjob': -4.448515892028809,
  'Ġuse': -4.654845237731934}]

## `pattern` keyword argument

The `pattern` argument is a regular expression that is used to contrain the text generated by the LM. When pattern is given only token that represent valid extensions of the pattern will be generated. This can be useful for enforcing formats (like only numbers). Just remember that the model does plan in advance for this contraint (yet), so you need to specify a format the model is already familar with.

In [6]:
# setting top_p to 0.4 will make the generated text less likely to chose low probability tokens by
# sampling from the top tokens that collectively have a probability of 0.4
out = guidance("""This is a sentence about {{gen 'text' stop=" " pattern="[0-9 ]+"}}""")()

# note that the odd character is due to the Transformers package tokenizer for GPT-2 representing the leading space as a special token
out["text"]

Stop program This is a sentence about 2

'2'

## `hidden` keyword argument

The `hidden` argument controls if the content generated will be included in the program contex (prompt) for future calls the language model.

In [1]:
import guidance

gpt2 = guidance.llms.Transformers("gpt2", device=1)
guidance.llm = gpt2

In [3]:
out = guidance("""This is a sentence about {{gen 'text1' stop=" " hidden=True}}{{gen 'text2' stop="."}}""")()

# notice that the first word is the same in both generated texts
# this is because the first generated text is hidden and not given as context for the second
out["text1"], out["text2"]

Stop program This is a sentence about how to make a good game

('how', 'how to make a good game')

## `parse` keyword argument

When the `parse` argument is True then the results of the generation will be recursively parsed as a guidance template.

In [3]:
out = guidance("""Write a story about a person and use \{{character_name}} whenever you need to write their name:
STORY
{{gen 'story' max_tokens=100 parse=True}}""", llm=gpt3, character_name="Jill")()

Stop program Write a story about a person and use {{character_name}} whenever you need to write their name:
STORY
 
 Jill was a young person with a lot of ambition. They had big dreams of making a difference in the world, but they weren't sure how to go about it.

One day, Jill decided to take a chance and apply for a job at a local non-profit organization. They were thrilled when they got the job and quickly got to work.

 Jill was passionate about the cause and worked hard to make a difference

## `list_append` keyword argument

When the `list_append` argument is True then the results of the generation will appended to the list given by the `name` argument. If not list exists with that name then a new list will be created. This can be a useful alternative to using the `geneach` command in some circumstances.

In [4]:
out = guidance("""Write three story title options about the arctic circle:
OUTLINE
1. "{{gen 'story' max_tokens=100 list_append=True}}"
2. "{{gen 'story' max_tokens=100 list_append=True}}"
3. "{{gen 'story' max_tokens=100 list_append=True}}"
""", llm=gpt3)()
out["story"]

Stop program Write three story title options about the arctic circle:
OUTLINE
1. " The Frozen North: A Journey Through the Arctic Circle "
2. " Exploring the Arctic: A Tale of Adventure and Discovery "
3. " The Icy Depths of the Arctic: A Voyage of Discovery "

['The Frozen North: A Journey Through the Arctic Circle',
 'Exploring the Arctic: A Tale of Adventure and Discovery',
 'The Icy Depths of the Arctic: A Voyage of Discovery']

: 

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>